In [142]:
import bs4
import pandas as pd

In [143]:
markup = open('ipcr_scheme_20150101.xml')
soup = bs4.BeautifulSoup(markup, 'lxml-xml')

In [144]:
KINDS = ['s', 'c', 'u', 'm', '1', '2', '3']
KINDS_dict = {'s': 'section', 'c': 'class', 'u': 'subclass', 'm': 'group'
              , '1': 'level 1', '2': 'level 2', '3': 'level 3'}

* s -> section
* c -> class
* u -> subclass
* m -> group
* 1 -> subgroup level 1
* 2 -> subgroup level 2
* 3 -> subgroup level 3

In [145]:
kind_s = soup.find_all('ipcEntry', kind=KINDS[0])

* `kind_s[0:8]` -> A-H in English
* `kind_s[8:]` -> A-H en francais

In [146]:
def kind_write(kind_list, kind_index, f):
    for kind in kind_list:
        text = '; '.join(list(kind.title.stripped_strings))
        line = str(kind['symbol']) + '& ' + str(KINDS_dict[KINDS[kind_index]]) + '& ' + text + '\n'
        f.write(line)
        if kind_index < len(KINDS) - 1:
            kind_new = kind.find_all('ipcEntry', kind=KINDS[kind_index + 1])
            if kind_new:
                kind_write(kind_new, kind_index + 1, f)

In [147]:
with open('ipc.csv', 'w') as f:
    kind_write(kind_s[0:8], 0, f)
with open('ipc_fr.csv', 'w') as f:
    kind_write(kind_s[8:], 0, f)

In [148]:
def kind_write2(kind_list, kind_index, f, kind_tree=None):
    for kind in kind_list:
        if kind_tree is None:
            kind_tree = []
            for _ in KINDS:
                kind_tree.append('')
        kind_tree[kind_index] = kind['symbol']
        for i in range(kind_index + 1, len(KINDS)):
            kind_tree[i] = ''
        text = '; '.join(list(kind.title.stripped_strings))
        line = '& '.join(kind_tree) + '& ' + text + '\n'
        f.write(line)
        if kind_index < len(KINDS) - 1:
            kind_new = kind.find_all('ipcEntry', kind=KINDS[kind_index + 1])
            if kind_new:
                kind_write2(kind_new, kind_index + 1, f, kind_tree=kind_tree)

In [149]:
with open('ipc2.csv', 'w') as f:
    columns = []
    for i in KINDS:
        columns.append(KINDS_dict[i])
    columns.append('title \n')
    f.write('& '.join(columns))
    kind_write2(kind_s[:8], 0, f)